# Feldspar Saturation Surface Calculator

In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
import sys
from thermoengine import core, phases, model, equilibrate

Allocate phase instances.  Note that liquid models may be selected corresponding to rhyolite-MELTS (v1.0), pMELTS or one of the two models developed by Ghiorso and Gualda (2015)

In [2]:
modelDB = model.Database(liq_mod='v1.0')
Liquid = modelDB.get_phase('Liq')
Feldspar = modelDB.get_phase('Fsp')
Water = phases.PurePhase('WaterMelts', 'H2O', calib=False)

### Composition of the liquid phase
Compsoition specified in grams of oxides. If these do not sum to 100, they will be normalized to wt%.

In [3]:
grm_oxides = {
    'SiO2':  77.5, 
    'TiO2':   0.08, 
    'Al2O3': 12.5, 
    'Fe2O3':  0.207,
    'Cr2O3':  0.0, 
    'FeO':    0.473, 
    'MnO':    0.0,
    'MgO':    0.03, 
    'NiO':    0.0, 
    'CoO':    0.0,
    'CaO':    0.43, 
    'Na2O':   3.98, 
    'K2O':    4.88, 
    'P2O5':   0.0, 
    'H2O':    5.5
}

Normalize this composition to 100%

In [4]:
sum_ox = sum(grm_oxides.values())
for key in grm_oxides.keys():
    grm_oxides[key] *= 100.0/sum_ox
print ('wt% H2O', round(grm_oxides['H2O'],3))

wt% H2O 5.209


Cast this composition as moles of eendmember liquid components; output as moles_end

In [5]:
mol_oxides = core.chem.format_mol_oxide_comp(grm_oxides, convert_grams_to_moles=True)
moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
if not Liquid.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")

### Feldspar-Liquid endmember reactions

Define a function that accepts an array of chemical potentials of endmember liquid components and returns a vector of chemical potentials of equivalent stoichiometry feldspar endmember components. Do this using explicit reference to endmember formulas, so that we do not have to remember or confuse indices that may change with alternate solution models. 

In [6]:
Feld_d = dict(zip(Feldspar.props['formula'], [i for i in range (0,Feldspar.props['endmember_num'])]))
Liq_d = dict(zip(Liquid.props['formula'], [i for i in range (0,Liquid.props['endmember_num'])]))
def reactions(t, p, muLiq):
    mu = np.zeros(Feldspar.props['endmember_num'])
    mu[Feld_d['NaAlSi3O8']] = muLiq[0,Liq_d['Na2SiO3']]/2.0 + muLiq[0,Liq_d['Al2O3']]/2.0 + 5.0*muLiq[0,Liq_d['SiO2']]/2.0
    mu[Feld_d['CaAl2Si2O8']] = muLiq[0,Liq_d['CaSiO3']] + muLiq[0,Liq_d['Al2O3']] + muLiq[0,Liq_d['SiO2']]
    mu[Feld_d['KAlSi3O8']] = muLiq[0,Liq_d['KAlSiO4']] + 2.0*muLiq[0,Liq_d['SiO2']]
    return np.array(mu)

To test the method, let's set a T (K) and P (bars) and calculate an array of Liquid chemical potentials

In [7]:
t = 1000
p = 1750
muLiq = Liquid.chem_potential(t, p, mol=moles_end)

Calculate from these liquid component chemical potentials equivalent potentials for feldspar stoichiometric endmembers.  Note that these result in chemical potentials of liquid components with feldspar stoichiomtry.  At equilibrium, these feldspar liquid chemical potentials would be equal to feldspar solid chemical potentials.

In [8]:
muFld = reactions(t, p, muLiq)

Now, translate these feldspar liquid chemical potentials into a solid feldspar affinity and composition.  If the affinity is negative, we have supersaturation, if postive undersaturation. The output composition array is in mole fractions of endmember feldspar components.

In [9]:
A, X = Feldspar.affinity_and_comp(t, p, muFld, debug=True, method='special')

Calling tailored Affinity and Comp routine for FeldsparBerman
... Affinity  -2322.199793102026 J/mol
... X [0.72059932 0.23541279 0.04398789]
... Convergence 1
... Iterations 12
... Affinity scalar 13.0
... Estimated error on affinity 0.02056985034778336


For this test case the affinity for water is (< 0 supersaturated, > 0 undersaturated) ...

In [11]:
Aw, Xw = Water.affinity_and_comp(t, p, np.array([muLiq[0,Liq_d['H2O']]]), debug=False)
print (round(Aw,3))

-488.501


## Find saturation state by finding $T_{sat}$
Create a function to use to zero the affinity as a function of T at fixed P and water content

In [13]:
def zero(t):
    global p, muFld, X
    A, X = Feldspar.affinity_and_comp(t, p, muFld, debug=False, method='special')
    return A

Next, use a root finder to solve for T when the affinity is zero.  This is $T_{sat}$.

In [15]:
sol = opt.root(zero, t)
print ('Saturated at', round(sol.x[0]-273.15,2),'°C')
print ('Composition:', X)

Saturated at 722.6 °C
Composition: [0.72066751 0.23636126 0.04297123]


## Find saturation state by varying wt% H<sub>2</sub>O
Create a function used to zero the affinity at fixed T and P by varying the water content

In [16]:
def zero_w(wtH2O):
    global t, p, X, grm_oxides
    new_sum = 100.0 - wtH2O
    old_sum = sum(grm_oxides.values()) - grm_oxides['H2O'] 
    new_oxides = {}
    for key in grm_oxides.keys():
        if key == 'H2O':
            new_oxides['H2O'] = wtH2O
        else:
            new_oxides[key] = grm_oxides[key]*new_sum/old_sum
    mol_oxides = core.chem.format_mol_oxide_comp(new_oxides, convert_grams_to_moles=True)
    moles_end,oxide_res = Liquid.calc_endmember_comp(
    mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
    muLiq = Liquid.chem_potential(t, p, mol=moles_end)
    muFld = reactions(t, p, muLiq)
    A, X = Feldspar.affinity_and_comp(t, p, muFld, debug=False, method='special')
    return A

Next, use a root finder to solve for wt% H<sub>2</sub>O at saturation

In [17]:
sol = opt.root(zero_w, grm_oxides['H2O'])
print ('Saturated at', round(sol.x[0],3),'Wt % H2O')
print ('Composition:', X)

Saturated at 6.806 Wt % H2O
Composition: [0.713344   0.22731096 0.05934504]
